In [1]:
!pip install -r requirements.txt -qq

#### **NBA DATASET AGENT**

**Set Up:**

Here we set up an environment for using Vertex AI and LangChain with an in-memory SQLite database. Here's a breakdown of its functionality:

1. **Loads environment variables**:
    - Reads environment variables from a file named `variables.env`.

2. **Sets up logging**:
    - Configures logging to display messages with a specific format and log level.

3. **Initializes Vertex AI**:
    - Sets up Vertex AI with the project ID and region specified in the environment variables.

4. **Configures a language model (LLM)**:
    - Sets up a language model (`ChatVertexAI`) with specific parameters like model name, temperature, and maximum output tokens.

5. **Creates an in-memory SQLite database**:
    - Establishes a connection to an in-memory SQLite database.

6. **Sets up short-term memory for agents**:
    - Initializes a `MemorySaver` instance for managing short-term memory.

In [2]:
import json
import logging
import os
from pathlib import Path

import pandas as pd
import sqlite3
import vertexai
from dotenv import load_dotenv
from langchain.agents import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_vertexai import ChatVertexAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Set up enviroment variables
dotenv_path = Path("variables.env")
load_dotenv(dotenv_path=dotenv_path)

project_id = os.getenv("PROJECT_ID")
region = os.getenv("REGION")

# Set up logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

# Create logger
_logger = logging.getLogger(__name__)

# Set up Vertex AI
_logger.info("Setting up Vertex AI")
vertexai.init(project=project_id, location=region)
_logger.info("Vertex AI set up")


# Set up LLMs
worker_llm = ChatVertexAI(
    model_name="gemini-1.5-pro",
    temperature=0.2,
    max_output_tokens=2048,
)

# Set up in-memory database
conn = sqlite3.connect(":memory:", check_same_thread=False)

# Set up agents short memory
memory = MemorySaver()
thread_id=0

2025-02-07 12:38:37,353 - __main__ - INFO - Setting up Vertex AI
2025-02-07 12:38:37,354 - __main__ - INFO - Vertex AI set up


### **Tools**

This Python script defines tools for managing CSV files and interacting with an SQLite database:

1. **List Files in CSV Folder**:
    - Lists all files in the 'csv' folder.
    - Returns filenames or an error message.

2. **List CSV Columns**:
    - Reads column names from a specified CSV file in the 'csv' folder.
    - Returns column names or an error message.

3. **Set Up Tables**:
    - Imports CSV files into the SQLite database as tables.
    - Returns a summary of successful and failed table setups.

4. **Select Info from CSV**:
    - Executes a SQL SELECT query on the database.
    - Returns query results as a dictionary or an error message.

These tools streamline file management and database operations for CSV data.

In [3]:
@tool
def list_files_in_csv_folder() -> list:
    """
    Lists all files in the 'csv' folder.

    Args:
        folder_path (str): Path to the folder.

    Returns:
        list: List of filenames in the folder.

    """
    try:
        files = os.listdir("csv")
        return [f for f in files if os.path.isfile(os.path.join("csv", f))]
    except Exception as e:
        return [f"Error: {e}"]

@tool
def list_csv_columns(filename: str) -> list:
    """
    Reads the column names of a selected CSV file from the 'csv' folder.

    Args:
        filename (str): The name of the CSV file (must exist in the 'csv' folder).

    Returns:
        list: A list of column names in the CSV file.

    """
    file_path = os.path.join("csv", filename)

    if not os.path.exists(file_path):
        return [f"Error: The file '{filename}' does not exist in the 'csv' folder."]

    try:
        df = pd.read_csv(file_path, nrows=1)  # Read only the header
        return df.columns.tolist()
    except Exception as e:
        return [f"Error: Could not read columns from '{filename}'. Reason: {e}"]

@tool
def set_up_tables(filenames: list[str]) -> str:
    """
    Set up SQL tables by importing CSV files into the database.

    For each file, it verifies that the file exists, reads the CSV data into a pandas DataFrame, and writes the data to an SQL table.
    Files that do not exist are recorded as failures.

    Args:
        filenames (list): A list of CSV filenames (including the .csv extension) to be processed.

    Returns:
        str: A summary message indicating the successful and failed table setups.

    """
    try:
        tables = [filename.strip(".csv") for filename in filenames]
        tables_failed=[]
        for filename in filenames:
            file_path = os.path.join("csv", filename)
            table_name = filename.strip(".csv")
            if not os.path.exists(file_path):
                tables_failed.append(filename)
            df = pd.read_csv(file_path)  # Read full CSV
            df.to_sql(table_name, conn, index=False, if_exists="replace")
        tables_successed=list(set(tables) - set(tables_failed))
    except Exception as e:
        return [f"Error: Could not set up tables '{tables}'. Reason: {e}"]
    else:
        return f"Successful tables set up: {tables_successed}, failed tables set up: {tables_failed}"


@tool
def select_info_from_csv(query: str) -> dict:
    """
    Execute a SQL SELECT query on the database and return the results.

    It only processes queries that begin with 'SELECT' (case-insensitive).

    Args:
        query (str): A SQL query string that should begin with 'SELECT' to retrieve data from the database.

    Returns:
        dict: A dict representation of the query results; otherwise, returns a string containing an error message.

    """
    try:
        if query.lower().startswith("select"):
            result_df = pd.read_sql(query, conn)
            return result_df.to_dict()
    except Exception as e:
        return f"Error: Query '{query}' could not be performed. Reason: {e}"
    else:
        return "Query must start with 'SELECT'"



### **Agent**

This Python script sets up a CSV explorer agent:

1. **System Prompt**:
    - Defines the agent as an NBA expert and SQLite database master.
    - Specifies the agent's tasks and SQL statement to check existing tables.

2. **CSV Explorer Tools**:
    - Lists the tools the agent will use: `list_files_in_csv_folder`, `list_csv_columns`, `set_up_tables`, and `select_info_from_csv`.

3. **Create Agent**:
    - Creates the CSV explorer agent using the specified model, tools, system prompt, and memory.

This setup enables the agent to analyze CSV data and interact with the SQLite database efficiently.

In [4]:
# Creates system prompt of CSV explorer agent
system_prompt_csv_explorer_agent = """
You are an NBA expert analyst and a database master specialized in sqlite3.
The data you work with is extracted from CSV files located in the 'csv' folder.
Rely on the column names to understand the structure of each table and identify potential relationships between them.
When obtaining the requested information, set up only the tables that are necessary, do not re-create tables that already exist.
Use the following SQL statement to check which tables are set up:
    SELECT name FROM sqlite_master WHERE type='table';
"""

# Indicates CSV explorer agent tools
csv_explorer_tools = [
    list_files_in_csv_folder,
    list_csv_columns,
    set_up_tables,
    select_info_from_csv
    ]

# Creates information extractor agent
csv_explorer_agent = create_react_agent(
    model=worker_llm,
    tools=csv_explorer_tools,
    state_modifier=SystemMessage(content=system_prompt_csv_explorer_agent),
    checkpointer=memory
)

This Python script sets up a conversational agent for interacting with CSV data:

1. **Safe Globals**:
    - Defines a dictionary to safely evaluate responses, disabling built-in functions for security.

2. **Main Function**:
    - Increments a global `thread_id` for each session.
    - Configures the agent with a unique thread ID.
    - Initiates a conversation loop where the agent responds to user questions.
    - Uses `csv_explorer_agent` to process user queries and generate responses.
    - Handles exceptions and logs errors.

This setup allows the agent to interact with users, process their questions, and provide responses based on CSV data.

In [5]:
safe_globals = {
    "AIMessage": lambda **kwargs: kwargs,
    "ToolMessage": lambda **kwargs: kwargs,
    "__builtins__": {}  # disable builtins to reduce risk
}

def main():
    global thread_id
    thread_id+=1
    try:
        config={"configurable": {"thread_id": f"nba{thread_id}"}}
        print("- NBA Expert: Hi, I'm here to help.")
        while True:
            print("- NBA Expert: Do you have any questions? Type 'No more questions' to end the conversation.")
            user_question = input("- Me:")
            if user_question.strip().lower() == "no more questions":
                print("- NBA Expert: Goodbye! 😊")
                break  # Exit the loop
            try:
                chunk_list = []
                for chunk in csv_explorer_agent.stream(
                    {"messages": [HumanMessage(content=user_question)]},config=config
                ):
                    response=eval(str(chunk),safe_globals)
                    _logger.info(f"{json.dumps(response, indent=4)}\n")
                    chunk_list.append(chunk)
                    if "agent" in chunk:
                        chunk_response = chunk["agent"]["messages"][0].content.encode().decode("utf-8")
                        if chunk_response:
                            print(f"- NBA Expert: {chunk_response}")
            except Exception:
                _logger.exception("An error occurred while processing your request")
    except KeyboardInterrupt:
        _logger.exception("Chatbot terminated by user.")
    except Exception:
        _logger.exception("Unexpected error")

# Usage
main()

- NBA Expert: Hi, I'm here to help.
- NBA Expert: Do you have any questions? Type 'No more questions' to end the conversation.


- Me: How many games were played in the 2012 season?  


2025-02-07 12:39:04,008 - __main__ - INFO - {
    "agent": {
        "messages": [
            {
                "content": "",
                "additional_kwargs": {
                    "function_call": {
                        "name": "list_files_in_csv_folder",
                        "arguments": "{}"
                    }
                },
                "response_metadata": {
                    "is_blocked": false,
                    "safety_ratings": [
                        {
                            "category": "HARM_CATEGORY_HATE_SPEECH",
                            "probability_label": "NEGLIGIBLE",
                            "probability_score": 0.158203125,
                            "blocked": false,
                            "severity": "HARM_SEVERITY_NEGLIGIBLE",
                            "severity_score": 0.1484375
                        },
                        {
                            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
             

- NBA Expert: There were 1420 games played in the 2012 season. 

- NBA Expert: Do you have any questions? Type 'No more questions' to end the conversation.


- Me: How many points per game averaged the team that plays in the 'TD Garden' arena (home and away games)?


2025-02-07 12:40:04,322 - __main__ - INFO - {
    "agent": {
        "messages": [
            {
                "content": "",
                "additional_kwargs": {
                    "function_call": {
                        "name": "list_csv_columns",
                        "arguments": "{\"filename\": \"teams.csv\"}"
                    }
                },
                "response_metadata": {
                    "is_blocked": false,
                    "safety_ratings": [
                        {
                            "category": "HARM_CATEGORY_HATE_SPEECH",
                            "probability_label": "NEGLIGIBLE",
                            "probability_score": 0.26953125,
                            "blocked": false,
                            "severity": "HARM_SEVERITY_LOW",
                            "severity_score": 0.22265625
                        },
                        {
                            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
 

- NBA Expert: The Boston Celtics, the team that plays in the TD Garden, averaged 101.78 points per game. 

- NBA Expert: Do you have any questions? Type 'No more questions' to end the conversation.


- Me: No more questions


- NBA Expert: Goodbye! 😊
